## CS2101 - Programming for Science and Finance
Prof. Götz Pfeiffer<br />
School of Mathematics, Statistics and Applied Mathematics<br />
University of Galway

***

### Digital Image Processing
# Week 6: Images

* **Digital images** are natural examples of collections of data.
* In fact, a digital image can be regarded as a matrix, or rather a **tensor, of numbers**.
* We'll discuss how this works and the tools in the `numpy` and `PIL` packages can be used to manipulate images, and to convert between **graphical** and **numerical representations** of the same data.
* First, we load the packages into the session.

In [ ]:
import numpy as np
print(np.__version__)
import PIL
print(PIL.__version__)
from PIL import Image

# Reading an image from a file

* Images often reside in files (as a sequence of $0$s and $1$s).
* One way to get an image into a Jupyter session is by using the `Image.open` function from the `PIL` library.
* The argument to `Image.open()` should be the file name if the file is in the same directory as the notebook. Otherwise, you will have to provide the full path to the file.

In [ ]:
img = Image.open('images/long_walk.png')
img

##  The RGB Color Model

* A digital image is a rectangular grid of **pixels**.
* Each pixel has one **intensity value** for each of the colours **red**, **green**, and **blue**.
* The **colour** of the pixel is determined by these three values.
* In NumPy, a digital image as a **three-dimensional array** of rows, columns, and colours.
* The entries in this array are either unsigned 8-bit integers, or floating point numbers in the interval $[0, 1]$.

* Let's convert the image to an array, and has a look inside.

In [ ]:
pic = np.asarray(img)

In [ ]:
print(pic)

* Three brackets at either end indicate that `pic` is a $3$-dimensional **tensor**.
* Let's check that `pic` is indeed a `numpy` array.

In [ ]:
type(pic)

* The entries themselves are of the type `uint8`, which means that they are unsigned 8-bit integers.
* These are the integers between $0$ and $255 = (2^8-1)$.
* **Warning**: All arithmetic on them is done modulo $256$.

In [ ]:
pic.dtype

In [ ]:
print(pic[0,0,0])
type(pic[0,0,0])

* We can see the modular artithmetic in action with some examples.
* This is a potential source of programming errors.

In [ ]:
a = pic[0,0,0]
b = pic[0,0,1]
print(a, b)
a, b

In [ ]:
print(a + b)
a + b

* We can check how many rows, columns, and colours the array has. 

In [ ]:
pic.shape

* The first dimension is the rows, the second is the columns, and the third is the [RGB](https://en.wikipedia.org/wiki/RGB_color_model) values of each pixel.

* PIL's `Image.fromarray` can convert the (numerical) array back into an image.

In [ ]:
Image.fromarray(pic)

* As `pic` is a numpy array, we can use `numpy` methods to manipulate the image.
* E.g., **slicing** the original produces a smaller picture.
* Here is all rows (in dim 0), columns 400 through 600 (in dim1) and all colours (in dim 3):

In [ ]:
smallpic = pic[:,400:601,:]
Image.fromarray(smallpic)

In [ ]:
smallpic.shape

* We will now see that we can make the image darker by multiplying each entry by a coefficient smaller than 1.
* When we multiply a `uint8` (or any integer) by a floating point value, the result is also a float, so we have to cast it back to a `uint8` after the multiplication.

In [ ]:
np.uint8(0.3 * pic[0,0,0])

In [ ]:
# change the intesity of the image by making it darker
coeff = 0.3
darker = smallpic.copy()
for i in range(darker.shape[0]):
    for j in range(darker.shape[1]):
        for k in range(darker.shape[2]):
            darker[i,j,k] = np.uint8(coeff * smallpic[i,j,k])

* Note how the 3 **nested** for loops make sure that every single entry of the array is multiplied by the same constant `coeff`.
* Also note how the `darker` array is first initialized as a copy of `smallpic` to provide the space for the darker pixels.
* Finally, the **type** `np.uint8` of the entries is used as a converion function.
* Let's see the resulting picture.

In [ ]:
Image.fromarray(darker)

* Numpy actually allows for a much quicker way to do this.  Quicker to write, and quicker to run!
* Instead of looping over all the pixels one by one, you can simply apply the operations to the whole matrix at once, like so:
  ```python
  np.uint8(coeff * smallpic)
  ```
*  Here, the time difference is perhaps not so noticable, but if we tried the nested loop on a larger picture with millions of pixels, it would take much longer than the quick way ...

In [ ]:
darker2 = np.uint8(coeff * smallpic)
Image.fromarray(darker2)

* We can change the intensity to make it lighter as well by multiplying by a coefficient larger than 1.

In [ ]:
# change the intesity of the image by making it lighter
coeff = 1.5
lighter0 = np.uint8(coeff * smallpic)
Image.fromarray(lighter0)

In [ ]:
np.uint8(coeff * smallpic)

* Oops ... what happened there?!

*  Before we convert the result of the multiplication back to a `uint8` we need to make sure that it is not greater than 255.
*  We can see a better way of doing this shortly, but for the time being we will use the nested loop approach. (Because applying the `min` function directly to the array does not work the way we want.)

In [ ]:
# change the intesity of the image by making it lighter
coeff = 1.5
lighter = smallpic.copy()
for i in range(lighter.shape[0]):
    for j in range(lighter.shape[1]):
        for k in range(lighter.shape[2]):
            lighter[i,j,k] = np.uint8(min(coeff * smallpic[i,j,k], 255))

In [ ]:
Image.fromarray(lighter)

## Separating Colors

* We can also see the different colour channels separately. To see e.g. the red separately, we set the blue and the green intensities to zero.

In [ ]:
r_pic = pic.copy()
r_pic[:,:,1:] = 0
Image.fromarray(r_pic)

In [ ]:
r_pic

* To see the blue channel, we set the red and the green to zero.

In [ ]:
b_pic = pic.copy()
b_pic[:,:,:2] = 0
Image.fromarray(b_pic)

In [ ]:
g_pic = pic.copy()
g_pic[:,:,::2] = 0
Image.fromarray(g_pic)

* The original image is the **sum** of its R, G and B colors.

In [ ]:
Image.fromarray(r_pic + b_pic + g_pic)

## Black Out

* We can change some portion of the image to black, by setting the RGB intensities to zero in some rectangular region.

In [ ]:
picwithsquare = pic.copy()
picwithsquare[50:150,400:500] = 0
Image.fromarray(picwithsquare)

## Flip

* Numpy provides a method `flip` to, well, flip an array with respect to a given dimension.

In [ ]:
# dimension 0 is the rows of the image
flip0 = np.flip(pic, 0)
Image.fromarray(flip0)

In [ ]:
# dimension 1 is the columns of the image
flip1 = np.flip(pic, 1)
Image.fromarray(flip1)

In [ ]:
# dimension 2 is the colors. What does it mean to flip them?
flip2 = np.flip(pic, 2)
Image.fromarray(flip2)

## Convex Combinations

* Like vectors and matrices, we can add arrays of the **same shape**.
* What does a sum mean, in terms of the corresponding images?

In [ ]:
smallpic2 = pic[:,500:701,:]
Image.fromarray(smallpic2)

In [ ]:
Image.fromarray(np.uint8(smallpic + smallpic2))

* Oops!
* Perhaps the average of two images works better ...

In [ ]:
Image.fromarray(np.uint8(0.5*smallpic + 0.5*smallpic2))

In [ ]:
smallpic3 = pic[:,405:606,:]
Image.fromarray(smallpic3)

In [ ]:
Image.fromarray(np.uint8(0.5*smallpic + 0.5*smallpic3))

### Convex sets

* A subset $X$ of $\mathbb{R}^n$ (for some $n$) is said to be **convex**, if for all $p,q \in X$, the line segment from $p$ to $q$ is also contained in $X$.
* If you think about this for a minute in 2 dimensions, it should be clear that this captures what we usually mean by 'convex' - something like 'no indentations'.

### Convex combinations

* The line segment from $p$ to $q$ consists of the points $(1-t)p + tq$ where $0\leq t \leq 1$. As a 2-dimensional example, let $p = (0,2)$ and $q = (3,1)$.
*  Then point halfway between $p$ and $q$ has coordinates
$$
0.5(0,2) + 0.5(3,1) = (1.5, 1.5).
$$
* The point three quarters of the way from $p$ to $q$ has coordinates
$$
0.25(0,2) + 0.75(3,1) = (2.25, 1.25).
$$

* If we have more points, say $p_1, p_2, p_3, \ldots, p_n$, then a _convex combination_ of $p_1, \ldots, p_n$ is a sum of the form
  $$
  \sum_{i=1}^n c_i p_i
  $$
  where $c_i \geq 0$ for $i=1,\ldots,n$ and $\sum c_i = 1$.

In [ ]:
Image.fromarray(np.uint8(0.6*pic + 0.4*flip0))

In [ ]:
Image.fromarray(np.uint8(0.3*pic + 0.7*flip2))

In [ ]:
Image.fromarray(np.uint8(0.1*pic + 0.2*flip0 + 0.3*flip1 + 0.4*flip2))

## Generating images

* We can generate images completely from scratch as well.
* As a first example, let's create a red circle on a black background.

* Create a black background to start with: a 500 by 500 pixel image, with the 3 colour channels.

In [ ]:
circpic = np.zeros((500,500,3), dtype=np.uint8)
Image.fromarray(circpic)

* Formula:  point $(x, y)$ is in the circle with **center** $(c, d)$ and **radius** $r$ if
  $$
  (x - c)^2 + (y - d)^2 \leq r^2
  $$

In [ ]:
for i in range(circpic.shape[0]):
    for j in range(circpic.shape[1]):
        # if we are at distance at most 100 from the point (200,350), make the pixel blue
        if (i-200)**2 + (j-350)**2 <= 100**2:
            circpic[i,j,2] = 200  # blue!
            
Image.fromarray(circpic)

* We can now add more than one circle, by defining the function `addcircle` that takes an x-coordinate and a y-coordinate for the centre of the circle as well as a value for the radius as parameters.

In [ ]:
def addcircle(arr, x, y, radius, blue):
    for r in range(arr.shape[0]):
        for c in range(arr.shape[1]):
            if (r-x)**2 + (c-y)**2 <= radius**2:
                arr[r,c,0] = blue

* Then we call that function repeatedly with random values for the coordinates and the radius.

In [ ]:
import random

In [ ]:
circpic2 = np.zeros((500,500,3), dtype=np.uint8)
for i in range(25):
    x = random.choice(range(500))
    y = random.choice(range(500))
    r = random.choice(range(60))
    c = random.choice(range(256))
    addcircle(circpic2, x, y, r, c)
                
Image.fromarray(circpic2)

## Summary

* Images really are 3D arrays of numbers.
* `numpy` can be used to efficiently manipulate images.

## References

### python

* The [`random` library](https://docs.python.org/3/library/random.html).

### numpy

* [flip](https://numpy.org/doc/stable/reference/generated/numpy.flip.html)
* [asarray](https://numpy.org/doc/stable/reference/generated/numpy.asarray.html)

### Other

* The Pillow [Tutorial](https://pillow.readthedocs.io/en/stable/handbook/tutorial.html)
* Matplotlib's [Image Tutorial](https://matplotlib.org/stable/tutorials/images.html).

## Exercises

* Load the image file `../aras_brun.png` and convert it into a numpy array of 8 bit unsigned integers.

In [ ]:
#  your code goes here